In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import datetime

In [2]:
df = pd.read_csv('spyData.csv')
options_df = pd.read_csv('options_df2.csv')

df.head()

,Unnamed: 0,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Dates,Jump
0,0,SPY,2010-01-04T08:00:00.000000000-05,112.12,112.18,111.44,112.14,1765282.0,428.0,2010-01-04,0
1,1,SPY,2010-01-04T08:15:00.000000000-05,112.14,112.16,112.09,112.15,682644.0,674.0,2010-01-04,0
2,2,SPY,2010-01-04T08:30:00.000000000-05,112.16,112.23,112.14,112.22,988952.0,873.0,2010-01-04,0
3,3,SPY,2010-01-04T08:45:00.000000000-05,112.22,112.33,112.22,112.32,378532.0,1167.0,2010-01-04,0
4,4,SPY,2010-01-04T09:00:00.000000000-05,112.33,112.46,112.33,112.39,554833.0,1641.0,2010-01-04,0


In [3]:
prices = df['Last']
log_prices = np.log(prices)
log_prices[0:5]

0    4.719748
1    4.719837
2    4.720461
3    4.721352
4    4.721975
Name: Last, dtype: float64

# New Lee-Mykland Calibration (2/7/24)

In [4]:
alpha_K = 0.5
K = 120

# Calibrating the Lee-Mykland Parameters

def calculate_sigma_hat_squared(log_prices, i, K):
    # Ensure we have enough data points to look back K periods from index i
    if i < K + 1:
        return np.nan  # return NaN if there isn't enough history
    # Calculate the sum of squared log returns over the window ending at index i
    sum_squared_log_returns = sum([
        np.abs(log_prices[j] - log_prices[j-1]) * np.abs(log_prices[j-1] - log_prices[j-2])
        for j in range(i-K+2, i)
    ])
    # Compute sigma_hat_squared
    sigma_hat_squared = (1 / (K - 2)) * sum_squared_log_returns
    return sigma_hat_squared

# Assuming the log_prices is a pandas Series, let's apply the function to each index
# We'll use a rolling apply to do this efficiently
sigma_hat_squared_series = pd.Series([calculate_sigma_hat_squared(log_prices, i, K) 
                                      for i in range(len(log_prices))])

L_i_series = pd.Series([(log_prices[i]-log_prices[i-1])/np.sqrt(sigma_hat_squared_series[i])
                       for i in range(K-1, len(sigma_hat_squared_series))])

In [5]:
alpha = 0.05

n = len(log_prices)
c = np.sqrt(2/np.pi)
C_n = (np.sqrt(2*np.log(n)))/c - (np.log(np.pi)+np.log(np.log(n)))/(2*c*np.sqrt(2*np.log(n)))
S_n = 1/(c*np.sqrt(2*np.log(n)))
df['Test Statistic'] = pd.Series([(np.abs(L_i_series[i]) - C_n)/S_n for i in range(len(L_i_series))])

Beta_star = -np.log(-np.log(1-alpha))
df['Jump'] = (df['Test Statistic'] > Beta_star).astype(int)

## Distribution of jumps

In [6]:
diffs = [i for i in df['Test Statistic'] if i > Beta_star] - Beta_star
# Define bucket size and range
bucket_size = 3
buckets = np.arange(0, max(diffs) + bucket_size, bucket_size)

# Group data into buckets
counts, bins = np.histogram(diffs, bins=buckets)

# Plot histogram
fig = go.Figure(data=[go.Bar(x=bins, y=counts, width=bucket_size, marker_color='steelblue')])
fig.update_layout(title=f'Differences from Threshold Beta* = {round(Beta_star, 2)}',
                  xaxis_title='Difference between threshold',
                  yaxis_title='Frequency',
                  xaxis=dict(tickvals=bins, ticktext=[f'{int(bins[i])}-{int(bins[i+1]-1)}' for i in range(len(bins)-1)]),
                  bargap=1)
fig.show()


In [7]:
jumps_df = df.loc[df['Jump']==1]
jumps_df = jumps_df.drop(columns = ['#RIC', 'Open', 'High', 'Low', 'Unnamed: 0'])
jumps_df['Difference'] = df['Test Statistic'] - Beta_star
jumps_df = jumps_df.sort_values(by=['Difference'])
jumps_df['Date-Time'].tail()

399      2010-01-15T17:45:00.000000000-05
15117    2011-07-05T08:45:00.000000000-04
7520     2010-10-01T08:00:00.000000000-04
7521     2010-10-01T08:15:00.000000000-04
3307     2010-05-03T14:45:00.000000000-04
Name: Date-Time, dtype: object

In [8]:
jumps_df

,Date-Time,Last,Volume,No. Trades,Dates,Jump,Test Statistic,Difference
8888,2010-11-18T10:00:00.000000000-05,120.1500,18572751.0,53715.0,2010-11-18,1,3.214018,0.243823
14875,2011-06-24T08:15:00.000000000-04,128.2200,657256.0,1649.0,2011-06-24,1,3.218426,0.248230
641,2010-01-27T08:15:00.000000000-05,109.3500,665212.0,1437.0,2010-01-27,1,3.285607,0.315412
19394,2011-12-05T09:00:00.000000000-05,126.5800,734681.0,1532.0,2011-12-05,1,3.342900,0.372705
7721,2010-10-08T08:15:00.000000000-04,116.1000,896478.0,1459.0,2010-10-08,1,3.403549,0.433354
...,...,...,...,...,...,...,...,...
399,2010-01-15T17:45:00.000000000-05,113.5600,151915.0,62.0,2010-01-15,1,51.438372,48.468177
15117,2011-07-05T08:45:00.000000000-04,133.7600,177912.0,633.0,2011-07-05,1,52.472553,49.502358
7520,2010-10-01T08:00:00.000000000-04,114.7100,1523500.0,405.0,2010-10-01,1,61.958031,58.987836
7521,2010-10-01T08:15:00.000000000-04,114.7400,764760.0,583.0,2010-10-01,1,78.148040,75.177845


# 5 most severe jumps:

## January 15, 2010, 5:45pm
## July 5, 2011, 8:45am
## October 1, 2010, 8:00am
## October 1, 2010, 8:15am
## May 3, 2010, 2:45pm

In [9]:
import plotly.express as px

# Assuming 'df' is your DataFrame and it contains the 'Date-Time' column in datetime format
# Convert 'Date-Time' to datetime format if it's not already
# df['Date-Time'] = pd.to_datetime(df['Date-Time'])

# Create a 'YearMonth' column by extracting year and month as strings and concatenating them
df['YearMonth'] = df['Date-Time'].apply(lambda x: f"{x.year}-{str(x.month).zfill(2)}")

# Group by 'YearMonth' and sum 'Jumps'
# monthly_jumps = df.groupby('YearMonth')['Jumps'].sum().reset_index()
monthly_jumps = df.groupby('YearMonth')['Jump'].sum().reset_index()


# Plot the histogram
fig = px.bar(monthly_jumps, x='YearMonth', y='Jump', title='Frequency of Jumps by Year and Month')
fig.update_xaxes(type='category')
fig.show()

AttributeError: 'str' object has no attribute 'year'

# There were 10-11 jumps in May 2010, Nov/Dec 2010, Aug 2011, Nov 2011.

## Possible Explanations:
### May 2010: Flash Crash
### Nov/Dec 2010 and Aug 2011: Concerns surrounding Euro sovereign debt and US debt ceiling
### Nov 2011: First ever time S&P downgraded the US credit rating from AAA to AA+

In [11]:
jump_points

,Unnamed: 0,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Dates,Jump,Test Statistic
43,43,SPY,2010-01-05T08:45:00.000000000-05,113.29,113.3598,113.26,113.2900,399719.0,895.0,2010-01-05,1,12.213564
121,121,SPY,2010-01-07T08:15:00.000000000-05,113.37,113.7300,113.31,113.3900,1242847.0,857.0,2010-01-07,1,4.367287
241,241,SPY,2010-01-12T08:15:00.000000000-05,113.98,113.9800,113.77,113.8000,1427735.0,3467.0,2010-01-12,1,7.868972
356,356,SPY,2010-01-14T17:00:00.000000000-05,115.10,115.1500,115.10,115.1200,77770.0,92.0,2010-01-14,1,3.406606
357,357,SPY,2010-01-14T17:15:00.000000000-05,115.12,115.1400,115.11,115.1200,9152.0,25.0,2010-01-14,1,3.789951
...,...,...,...,...,...,...,...,...,...,...,...,...
19547,19548,SPY,2011-12-08T17:15:00.000000000-05,123.88,123.9100,123.76,123.7600,122454.0,90.0,2011-12-08,1,25.810740
19549,19550,SPY,2011-12-08T17:45:00.000000000-05,123.88,123.9500,123.85,123.9500,129269.0,206.0,2011-12-08,1,20.815127
19699,19700,SPY,2011-12-14T15:15:00.000000000-05,121.94,122.0000,121.65,121.9654,12660982.0,32116.0,2011-12-14,1,4.782427
19711,19712,SPY,2011-12-15T08:15:00.000000000-05,122.67,122.8100,122.62,122.7500,282106.0,570.0,2011-12-15,1,14.824475


In [10]:
# Convert the 'Date-Time' column to datetime
#df['Date-Time'] = pd.to_datetime(df['Date-Time'])

# Create a scatter trace for the 'Last' values
trace0 = go.Scatter(
    x=df['Date-Time'],
    y=df['Last'],
    mode='lines',
    name='Price'
)

# Create a scatter trace for the jumps, where 'jump' equals 1
jump_points = df[df['Jump'] == 1]
trace1 = go.Scatter(
    x=jump_points['Date-Time'],
    y=jump_points['Last'],
    mode='markers',
    name='Jumps',
    marker=dict(color='red')
)

# Define the layout for the plot
layout = go.Layout(
    title='Last vs Date-Time with Jumps',
    xaxis=dict(title='Date-Time'),
    yaxis=dict(title='Last'),
    showlegend=True
)

# Create the figure with both traces
fig = go.Figure(data=[trace0, trace1], layout=layout)

# Display the figure
fig.show()

### 173 total jumps at the 5% significance level.

# Old "Lee-Mykland" calibration (Dec 2023) - similar but not exact

In [12]:
df['log_return'] = np.log(df['Last'] / df['Last'].shift(1))

# Drop the NaN values created by the shift operation
df = df.dropna()

# Set the window size K for calculating the rolling standard deviation of log returns
# Assuming K is given in the paper or can be chosen appropriately
K = 60  # Example window size, the appropriate value should be chosen based on the research paper

# Calculate the rolling standard deviation of log returns
df['rolling_std'] = df['log_return'].rolling(window=K).std()

# Calculate the test statistic L(i) for each point in time
df['L'] = df['log_return'] / df['rolling_std']

# Choose a significance level α (e.g., a5%)
alpha = 0.05

# Calculate the threshold value for jump detection
# Assuming the formula for the threshold is given in the paper and using norm.ppf for the inverse cumulative distribution function
# Adjust the formula according to the one provided in the paper
threshold = -np.log(-np.log(1 - alpha))

# Identify jumps by comparing the absolute value of L(i) with the threshold
df['jump'] = (df['L'].abs() > threshold).astype(int)

# Count the number of jumps detected
num_jumps = df['jump'].sum()

num_jumps

376

In [13]:
jumps = np.where(df['jump']==1)[0].tolist() #indexes of jumps

In [14]:
df['Return'] = np.log1p(df['Last'].pct_change())

In [15]:
df.head()

,Unnamed: 0,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Dates,Jump,Test Statistic,log_return,rolling_std,L,jump,Return
2,2,SPY,2010-01-04T08:30:00.000000000-05,112.16,112.23,112.14,112.22,988952.0,873.0,2010-01-04,0,-17.089107,0.000624,NaN,NaN,0,NaN
3,3,SPY,2010-01-04T08:45:00.000000000-05,112.22,112.33,112.22,112.32,378532.0,1167.0,2010-01-04,0,-13.543801,0.000891,NaN,NaN,0,0.000891
4,4,SPY,2010-01-04T09:00:00.000000000-05,112.33,112.46,112.33,112.39,554833.0,1641.0,2010-01-04,0,-18.102734,0.000623,NaN,NaN,0,0.000623
5,5,SPY,2010-01-04T09:15:00.000000000-05,112.39,112.44,111.44,112.36,684084.0,2020.0,2010-01-04,0,-17.083539,-0.000267,NaN,NaN,0,-0.000267
6,6,SPY,2010-01-04T09:30:00.000000000-05,112.37,112.75,112.33,112.73,12623404.0,31193.0,2010-01-04,0,-17.588105,0.003288,NaN,NaN,0,0.003288


In [16]:
jump_returns = df.loc[df['jump'] == 1, 'Return'].tolist()

In [17]:
positive = 0
negative = 0
pos_jumps = []
neg_jumps = []

for jump in jump_returns:
    
    if jump > 0:
        positive += 1
        pos_jumps.append(jump)
    
    if jump < 0:
        negative += 1
        neg_jumps.append(jump)

In [18]:
np.mean(neg_jumps)
np.std(pos_jumps)

0.006444213462595526

In [19]:
import numpy as np
from scipy.stats import norm

def kou_jump_diffusion(S, K, r, q, T, sigma, lambda_, mu, delta, p):
    """
    Kou double exponential jump diffusion option pricing model.

    Parameters:
        S (float): Initial stock price
        K (float): Strike price
        r (float): Risk-free interest rate
        q (float): Dividend yield
        T (float): Time to maturity
        sigma (float): Volatility of the underlying asset
        lambda_ (float): Intensity of the jump process
        mu (float): Expected jump size
        delta (float): Standard deviation of jump size
        p (float): Probability of an up jump

    Returns:
        call_price (float): Price of a European call option
        put_price (float): Price of a European put option
    """
    N = 1000  # Number of simulations
    dt = T / 100  # Time step
    sqrt_dt = np.sqrt(dt)

    # Simulate stock price paths
    ST = S * np.exp((r - q - (0.5 * (sigma ** 2))) * T +
                    sigma * np.random.randn(N) * sqrt_dt)
    
    # Generate jump components
    jump = np.random.poisson(lambda_ * T, N)
    jump_size = np.random.normal(mu, delta, N)
    for i in range(N):
        for j in range(jump[i]):
            if np.random.rand() <= p:
                ST[i] *= np.exp(jump_size[i])
            else:
                ST[i] *= np.exp(-jump_size[i])

    # Calculate option payoffs
    call_payoff = np.maximum(ST - K, 0)
    put_payoff = np.maximum(K - ST, 0)

    # Discounted expected payoffs
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    put_price = np.exp(-r * T) * np.mean(put_payoff)

    return call_price, put_price

# Example usage
S = 100  # Initial stock price
K = 100  # Strike price
r = 0.05  # Risk-free interest rate
q = 0.03  # Dividend yield
T = 1  # Time to maturity
sigma = 0.2  # Volatility
lambda_ = 0.1  # Intensity of the jump process
mu = 0.1  # Expected jump size
delta = 0.1  # Standard deviation of jump size
p = 0.5  # Probability of an up jump

call_price, put_price = kou_jump_diffusion(S, K, r, q, T, sigma, lambda_, mu, delta, p)
print("European Call Price:", call_price)
print("European Put Price:", put_price)

European Call Price: 1.1407829403487353
European Put Price: 1.1338248759943859


In [20]:
def blackScholes(S0, sig, T, t, K, r, div, option_type):
    d_plus = (np.log(S0/K) + (r - div + (sig**2)/2)*(T-t))/(sig*np.sqrt(T-t))
    d_minus = d_plus - (sig*np.sqrt(T-t))
    
    if option_type == "call":
        option_price = S0*np.exp(-div*(T-t))*norm.cdf(d_plus) - (K*np.exp(-r*(T-t)))*norm.cdf(d_minus)
        
    if option_type == "put":
        option_price = (K*np.exp(-r*(T-t)))*norm.cdf(-d_minus) - S0*np.exp(-div*(T-t))*norm.cdf(-d_plus)
        
    return option_price

In [24]:
# Example usage
S = 100  # Initial stock price
K = 100  # Strike price
r = 0.05  # Risk-free interest rate
# q = 0.015  # Dividend yield
q = 0
T = 1  # Time to maturity
sigma = 0.2  # Volatility
lambda_ = 0.05  # Intensity of the jump process
lambda_ = len(all_jumps)/len(df)
mu = np.mean(pos_jumps)  # Expected jump size
delta = np.std(pos_jumps)  # Standard deviation of jump size
p = 0.5  # Probability of an up jump

call_price, put_price = kou_jump_diffusion(S, K, r, q, T, sigma, lambda_, mu, delta, p)
print("European Call Price:", call_price)
print("European Put Price:", put_price)

European Call Price: 2.955218042277972
European Put Price: 0.07052341844833115


In [26]:
print("Black Scholes price call: ", blackScholes(S, sigma, T, 0, K, r, q, "call"))
print("Black Scholes price put: ", blackScholes(S, sigma, T, 0, K, r, q, "put"))

Black Scholes price call:  10.450583572185565
Black Scholes price put:  5.573526022256971


In [22]:
all_jumps = pos_jumps + neg_jumps
len(all_jumps)/len(df)
np.std(np.abs(all_jumps))
# len(pos_jumps)
# len(neg_jumps)

0.006365984471408831

In [27]:
all_jumps = [*pos_jumps, *neg_jumps]


In [28]:
mu2 = np.mean(np.abs(all_jumps))
delta2 = np.std(np.abs(all_jumps))
delta2
np.std(pos_jumps)

0.006444213462595526

In [143]:
paths = 1000
m = 25*252
S0 = 100
deltaT = T/m
p = 0.5

def kou_pricing(S0, K, r, q, T, sigma, lambda_, mu, delta, p, paths, option_type):

    S_lst = [None]*paths
    V_lst = [None]*paths

    m = int(26*T*252)
    if m == 0:
        deltaT = 0.0000000000001
    else:
        deltaT = T/m

    # V_lst_call = [None]*paths
    # V_lst_put = [None]*paths

    jump = np.random.poisson(lambda_ * T, [paths, m])
    jump_size = (np.random.laplace(mu, delta, [paths, m]))
    for j in range(paths):
        S_lst[j] = [None]*m
        S_lst[j][0] = S0
        for i in range(1, m):
            S_lst[j][i] = S_lst[j][i-1]*(1 + r*deltaT + sigma*np.sqrt(deltaT)*np.random.normal(0,1))

            if jump[j][i] > 0:
                if np.random.rand() <= p:
                    S_lst[j][i] *= np.exp(jump_size[j][i])
                else:
                    S_lst[j][i] *= np.exp(-jump_size[j][i])

        
        if option_type == "call":
            V_lst[j] = max((S_lst[j][-1] - K), 0)
        else:
            V_lst[j] = max((K - S_lst[j][-1]), 0)

        # V_lst_call[j] = max((S_lst[j][-1] - K), 0)
        # V_lst_put[j] = max((K - S_lst[j][-1]), 0)

    return np.mean(V_lst)

# print("Kou call price: ", np.mean(V_lst_call))
# print("Kou put price: ", np.mean(V_lst_put))

In [31]:
options_df.head()

,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Root,Strike,Expiry,Option Type
0,SPYA161010000.U,2010-01-04T09:30:00.000000000-05,12.50,12.75,12.50,12.75,19.0,3.0,SPY,100.0,2010-01-16,call
1,SPYA161010200.U,2010-01-04T09:30:00.000000000-05,10.84,10.84,10.84,10.84,40.0,1.0,SPY,102.0,2010-01-16,call
2,SPYA161010400.U,2010-01-04T09:30:00.000000000-05,8.48,8.48,8.47,8.47,2.0,2.0,SPY,104.0,2010-01-16,call
3,SPYA161010500.U,2010-01-04T09:30:00.000000000-05,7.65,7.88,7.65,7.88,84.0,10.0,SPY,105.0,2010-01-16,call
4,SPYA161010600.U,2010-01-04T09:30:00.000000000-05,6.80,6.89,6.80,6.83,13.0,4.0,SPY,106.0,2010-01-16,call


In [131]:
for idx in (options_df[options_df["Date-Time"] == df["Date-Time"][7]]).index:
    print(idx)

169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327


In [125]:
df.loc[18010]

Unnamed: 0                                   18011
#RIC                                           SPY
Date-Time         2011-10-14T12:00:00.000000000-04
Open                                       121.371
High                                         121.5
Low                                         121.28
Last                                        121.31
Volume                                   3845125.0
No. Trades                                 13211.0
Dates                                   2011-10-14
Jump                                             0
Test Statistic                           -8.663669
log_return                               -0.000503
rolling_std                               0.001959
L                                        -0.256642
jump                                             0
Return                                   -0.000503
Name: 18010, dtype: object

In [109]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(options_df[options_df["Date-Time"] == df["Date-Time"][6]])
    display(options_df[options_df["Date-Time"] == df["Date-Time"][10000]])

,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Root,Strike,Expiry,Option Type


348

In [43]:
options_idx = 152
S0 = df["Last"][6]
K = options_df["Strike"][options_idx]
r = 0.05
q = 0
T = days_to_maturity[options_idx]/365
sigma = np.std(df["Return"])*np.sqrt(26*252)
lambda_ = len(all_jumps)/len(df)
mu = np.mean(np.abs(all_jumps))
delta = np.std(np.abs(all_jumps))
p = len(pos_jumps)/len(all_jumps)
paths = 1000
option_type = options_df["Option Type"][options_idx]
market_price = options_df["Last"][options_idx]

In [44]:
print("Jump Diffusion price: ", kou_pricing(S0, K, r, q, T, sigma, lambda_, mu, delta, p, paths, option_type))
print("Black Scholes price: ", blackScholes(S0, sigma, T, 0, K, r, q, option_type))
print("Market price: ", market_price)

Jump Diffusion price:  5.238116562084153
Black Scholes price:  3.7193944236795033
Market price:  6.73


In [45]:
kou_diffs = []
bs_diffs = []
for idx in (options_df[options_df["Date-Time"] == df["Date-Time"][100]]).index:
    K = options_df["Strike"][idx]
    T = days_to_maturity[idx]/365
    option_type = options_df["Option Type"][idx]
    market_price = options_df["Last"][idx]

    kou_price = kou_pricing(S0, K, r, q, T, sigma, lambda_, mu, delta, p, paths, option_type)
    bs_price = blackScholes(S0, sigma, T, 0, K, r, q, option_type)

    kou_diff = np.abs((kou_price-market_price)/market_price)
    bs_diff = np.abs((bs_price-market_price)/market_price)

    kou_diffs.append(kou_diff)
    bs_diffs.append(bs_diff)



In [79]:
lst = [[] for _ in range(len(dt_idxs))]
lst[0].append(1)
lst


[[1], [], [], []]

In [132]:
dt_idxs = [100, 1010, 10100, 18010]
kou_diffs = [[] for _ in range(len(dt_idxs))]
bs_diffs = [[] for _ in range(len(dt_idxs))]
counter = 0

for dt_idx in dt_idxs:
    for idx in (options_df[options_df["Date-Time"] == df["Date-Time"][dt_idx]]).index:
        K = options_df["Strike"][idx]
        T = days_to_maturity[idx]/365
        option_type = options_df["Option Type"][idx]
        market_price = options_df["Last"][idx]

        kou_price = kou_pricing(S0, K, r, q, T, sigma, lambda_, mu, delta, p, paths, option_type)
        bs_price = blackScholes(S0, sigma, T, 0, K, r, q, option_type)

        kou_diff = np.abs((kou_price-market_price)/market_price)
        bs_diff = np.abs((bs_price-market_price)/market_price)

        kou_diffs[counter].append(kou_diff)
        bs_diffs[counter].append(bs_diff)

    counter += 1

ZeroDivisionError: float division by zero

In [133]:
kou_mean_lst = []
bs_mean_lst = []
for i in range(len(dt_idxs)):
    kou_mean_lst.append(np.mean(kou_diffs[i]))
    bs_mean_lst.append(np.mean(bs_diffs[i]))

In [137]:
len(bs_diffs[0])
len((options_df[options_df["Date-Time"] == df["Date-Time"][100]]).index)
np.mean(kou_diffs[0])

0.3308646409947053

In [140]:
print(kou_mean_lst)
print(bs_mean_lst)

[0.3308646409947053, 1.9550771505709623, 8.034427221380742, 0.5620749198921113]
[0.33407868373186744, 1.951155068656959, 7.8508868073245965, 0.5785752056470753]


In [93]:
#dt_idxs = [100, 1000, 10000, 15000]
print(kou_mean_lst)
print(bs_mean_lst)
print(np.mean(kou_mean_lst))
print(np.mean(bs_mean_lst))

[0.31367078398473736, nan, nan, 21.727863811297286]
[0.33407868373186744, nan, nan, 21.59139308110215]
nan
nan


In [46]:
#using df["Date-Time"][100]
print(np.mean(kou_diffs))
print(np.mean(bs_diffs))

0.3249317542905543
0.33407868373186744


In [150]:
#using df["Date-Time"][6]
print(np.mean(kou_diffs))
print(np.mean(bs_diffs))

0.4467634339837234
0.5063135483949364


In [30]:
paths = 1000
m = 25*252
S0 = 100
deltaT = T/m
p = 0.5

S_lst = [None]*paths
V_lst_call = [None]*paths
V_lst_put = [None]*paths

jump = np.random.poisson(lambda_ * T, [paths, m])
# jump_size = np.abs(np.random.normal(mu, delta, [paths, m]))
jump_size = (np.random.normal(mu, delta, [paths, m]))
for j in range(paths):
    S_lst[j] = [None]*m
    S_lst[j][0] = S0
    for i in range(1, m):
        S_lst[j][i] = S_lst[j][i-1]*(1 + r*deltaT + sigma*np.sqrt(deltaT)*np.random.normal(0,1))

        # if jump[j][i] > 0:
        #     if np.random.rand() <= p:
        #         S_lst[j][i] *= np.exp(jump_size[j][i])
        #     else:
        #         S_lst[j][i] *= np.exp(-jump_size[j][i])

    
    V_lst_call[j] = max((S_lst[j][-1] - K), 0)
    V_lst_put[j] = max((K - S_lst[j][-1]), 0)

print("Kou call price: ", np.mean(V_lst_call))
print("Kou put price: ", np.mean(V_lst_put))

Kou call price:  11.099961718607322
Kou put price:  6.2356974622726975


In [29]:
jump = np.random.poisson(lambda_ * T, [paths, m])
jump


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [38]:
df

,Unnamed: 0,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Dates,Jump,Test Statistic,log_return,rolling_std,L,jump,Return
2,2,SPY,2010-01-04T08:30:00.000000000-05,112.16,112.23,112.14,112.22,988952.0,873.0,2010-01-04,0,-17.089107,0.000624,NaN,NaN,0,NaN
3,3,SPY,2010-01-04T08:45:00.000000000-05,112.22,112.33,112.22,112.32,378532.0,1167.0,2010-01-04,0,-13.543801,0.000891,NaN,NaN,0,0.000891
4,4,SPY,2010-01-04T09:00:00.000000000-05,112.33,112.46,112.33,112.39,554833.0,1641.0,2010-01-04,0,-18.102734,0.000623,NaN,NaN,0,0.000623
5,5,SPY,2010-01-04T09:15:00.000000000-05,112.39,112.44,111.44,112.36,684084.0,2020.0,2010-01-04,0,-17.083539,-0.000267,NaN,NaN,0,-0.000267
6,6,SPY,2010-01-04T09:30:00.000000000-05,112.37,112.75,112.33,112.73,12623404.0,31193.0,2010-01-04,0,-17.588105,0.003288,NaN,NaN,0,0.003288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20026,20027,SPY,2011-12-27T17:00:00.000000000-05,126.38,126.39,126.36,126.38,5570.0,16.0,2011-12-27,0,-13.280174,-0.000079,0.000724,-0.109241,0,-0.000079
20027,20028,SPY,2011-12-27T17:15:00.000000000-05,126.37,126.58,126.35,126.35,619026.0,46.0,2011-12-27,0,-14.398788,-0.000237,0.000725,-0.327396,0,-0.000237
20028,20029,SPY,2011-12-27T17:30:00.000000000-05,126.49,126.49,126.34,126.36,17043.0,8.0,2011-12-27,0,-16.998567,0.000079,0.000724,0.109373,0,0.000079
20029,20030,SPY,2011-12-27T17:45:00.000000000-05,126.34,126.36,126.34,126.36,1829.0,7.0,2011-12-27,0,-16.264818,0.000000,0.000723,0.000000,0,0.000000


In [37]:
np.std(df["Return"])*np.sqrt(26*252)

0.17347810277666942

In [41]:
df.head()

,Unnamed: 0,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Dates,Jump,Test Statistic,log_return,rolling_std,L,jump,Return
2,2,SPY,2010-01-04T08:30:00.000000000-05,112.16,112.23,112.14,112.22,988952.0,873.0,2010-01-04,0,-17.089107,0.000624,NaN,NaN,0,NaN
3,3,SPY,2010-01-04T08:45:00.000000000-05,112.22,112.33,112.22,112.32,378532.0,1167.0,2010-01-04,0,-13.543801,0.000891,NaN,NaN,0,0.000891
4,4,SPY,2010-01-04T09:00:00.000000000-05,112.33,112.46,112.33,112.39,554833.0,1641.0,2010-01-04,0,-18.102734,0.000623,NaN,NaN,0,0.000623
5,5,SPY,2010-01-04T09:15:00.000000000-05,112.39,112.44,111.44,112.36,684084.0,2020.0,2010-01-04,0,-17.083539,-0.000267,NaN,NaN,0,-0.000267
6,6,SPY,2010-01-04T09:30:00.000000000-05,112.37,112.75,112.33,112.73,12623404.0,31193.0,2010-01-04,0,-17.588105,0.003288,NaN,NaN,0,0.003288


In [40]:
options_df = pd.read_csv('options_df2.csv')

options_df.head()

,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Root,Strike,Expiry,Option Type
0,SPYA161010000.U,2010-01-04T09:30:00.000000000-05,12.50,12.75,12.50,12.75,19.0,3.0,SPY,100.0,2010-01-16,call
1,SPYA161010200.U,2010-01-04T09:30:00.000000000-05,10.84,10.84,10.84,10.84,40.0,1.0,SPY,102.0,2010-01-16,call
2,SPYA161010400.U,2010-01-04T09:30:00.000000000-05,8.48,8.48,8.47,8.47,2.0,2.0,SPY,104.0,2010-01-16,call
3,SPYA161010500.U,2010-01-04T09:30:00.000000000-05,7.65,7.88,7.65,7.88,84.0,10.0,SPY,105.0,2010-01-16,call
4,SPYA161010600.U,2010-01-04T09:30:00.000000000-05,6.80,6.89,6.80,6.83,13.0,4.0,SPY,106.0,2010-01-16,call


In [93]:
options_df.tail()

,#RIC,Date-Time,Open,High,Low,Last,Volume,No. Trades,Root,Strike,Expiry,Option Type
2587552,SPYX311213000.U,2011-12-30T16:00:00.000000000-05,15.10,15.10,15.10,15.10,10.0,1.0,SPY,130.0,2012-12-31,put
2587553,SPYA211212900.U,2011-12-30T16:15:00.000000000-05,0.92,0.92,0.92,0.92,3500.0,2.0,SPY,129.0,2012-01-21,call
2587554,SPYM211211800.U,2011-12-30T16:15:00.000000000-05,0.56,0.56,0.56,0.56,6500.0,2.0,SPY,118.0,2012-01-21,put
2587555,SPYM211212500.U,2011-12-30T16:15:00.000000000-05,2.11,2.11,2.11,2.11,6500.0,2.0,SPY,125.0,2012-01-21,put
2587556,SPYX301112700.U,2011-12-30T16:15:00.000000000-05,1.25,1.25,1.25,1.25,3500.0,2.0,SPY,127.0,2011-12-30,put


In [38]:
options_df["Expiry"] = pd.to_datetime(options_df["Expiry"])
options_df["Expiry"]

0         2010-01-16
1         2010-01-16
2         2010-01-16
3         2010-01-16
4         2010-01-16
             ...    
2587552   2012-12-31
2587553   2012-01-21
2587554   2012-01-21
2587555   2012-01-21
2587556   2011-12-30
Name: Expiry, Length: 2587557, dtype: datetime64[ns]

In [39]:
dates = [None] * len(options_df)
for i in range(len(options_df)):
    dates[i] = options_df["Date-Time"][i][0:10]



In [40]:
options_df["Date-Time"]

0          2010-01-04T09:30:00.000000000-05
1          2010-01-04T09:30:00.000000000-05
2          2010-01-04T09:30:00.000000000-05
3          2010-01-04T09:30:00.000000000-05
4          2010-01-04T09:30:00.000000000-05
                         ...               
2587552    2011-12-30T16:00:00.000000000-05
2587553    2011-12-30T16:15:00.000000000-05
2587554    2011-12-30T16:15:00.000000000-05
2587555    2011-12-30T16:15:00.000000000-05
2587556    2011-12-30T16:15:00.000000000-05
Name: Date-Time, Length: 2587557, dtype: object

In [41]:
dates_lst = [datetime.datetime.strptime(date, "%Y-%m-%d").date() for date in dates]
expiry_dates_lst = [datetime.datetime.strptime(date, "%Y-%m-%d").date() for date in options_df["Expiry"].astype(str)]

In [42]:
days_to_maturity = [None] * len(dates_lst)
for i in range(len(dates_lst)):
    days_to_maturity[i] = (expiry_dates_lst[i] - dates_lst[i]).days